# Processing TAQ Data
Example of how to process a collection of csv files, that are compressed, and arranged in a bespoke directory structure. The structure is **ROOT**/**DATE**/**A-Z First Letter of Ticker**/**TICKER**.csv.gz. All files come from and S3 bucket.

## Algoseek LLC Data
Trade and Quote data has been provided by [AlgoSeek LLC](https://www.algoseek.com/), you can learn more about their data offerings from their home page.


In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import os
import subprocess
import boto3
import json
import datetime

from env import *
import pykx as kx
import awswrangler as wr

from managed_kx import *

In [2]:
# ----------------------------------------------------------------
DB_NAME="DEMO_DB"
DBVIEW_NAME=f"{DB_NAME}_VIEW"
SCALING_GROUP_NAME="DEMO_SCALING_GROUP"
VOLUME_NAME="DEMO_SHARED_VOLUME"
CODEBASE="demo"
CLUSTER_NAME="demo_csv_cluster"

HDB_CLUSTER_NAME="demo_hdb_cluster"

# S3 Destinations
S3_CODE_PATH="code"
S3_DATA_PATH="data"
SOURCE_DATA_DIR="demo"
# ----------------------------------------------------------------
WORKING_DIR=f"/opt/kx/app/shared/{VOLUME_NAME}/{CLUSTER_NAME}"

LOCAL_DATA_HOME="algoseek-marketdata/us-equity-taq-faang"
S3_DATA_HOME="s3://kdb-demo-829845998889-kms/algoseek-marketdata/us-equity-taq-faang"

# Two days supplied in Tarball
DATE=datetime.date(2021,1,4)
#DATE=datetime.date(2021,1,5)

# ----------------------------------------------------------------

# set q console width and height
kx.q("\c 200 200")

pykx.Identity(pykx.q('::'))

In [3]:
# Create AWS Session for working with FinSpace service
session=None

try:
    # aws: use ada for credentials
    os.system(["which", "ada"])
    os.system(f"ada credentials update --account={ACCOUNT_ID} --provider=isengard --role=Admin --once")
except: 
    None

if AWS_ACCESS_KEY_ID is None:
    print("Using Defaults ...")
    # create AWS session: using access variables
    session = boto3.Session()
else:
    print("Using variables ...")
    session = boto3.Session(
        aws_access_key_id=AWS_ACCESS_KEY_ID,
        aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
        aws_session_token=AWS_SESSION_TOKEN
    )

# create finspace client
client = session.client(service_name='finspace', endpoint_url=ENDPOINT_URL)

Using Defaults ...


# Stage TAQ Data to S3
Copy the supplied sample TAQ data to an S3 bucket. Then the data will be processed by GP cluster from the S3 location. 

In [4]:
!rm -rf algoseek-marketdata

In [9]:
!tar xzf algoseek-marketdata.tar.gz

In [10]:
# Stage TAQ data to S3
if AWS_ACCESS_KEY_ID is not None:
    cp = f"""
export AWS_ACCESS_KEY_ID={AWS_ACCESS_KEY_ID}
export AWS_SECRET_ACCESS_KEY={AWS_SECRET_ACCESS_KEY}
export AWS_SESSION_TOKEN={AWS_SESSION_TOKEN}

aws s3 rm --recursive {S3_DATA_HOME}
aws s3 sync --exclude .DS_Store {LOCAL_DATA_HOME} {S3_DATA_HOME}
aws s3 ls {S3_DATA_HOME}
"""
else:
    cp = f"""
aws s3 rm --recursive {S3_DATA_HOME}
aws s3 sync --exclude .DS_Store {LOCAL_DATA_HOME} {S3_DATA_HOME}
aws s3 ls {S3_DATA_HOME}
"""
    
# execute the S3 copy
os.system(cp)

upload: algoseek-marketdata/us-equity-taq-faang/2021/20210104/A/AMZN.csv.gz to s3://kdb-demo-829845998889-kms/algoseek-marketdata/us-equity-taq-faang/2021/20210104/A/AMZN.csv.gz
upload: algoseek-marketdata/us-equity-taq-faang/2021/20210104/F/FB.csv.gz to s3://kdb-demo-829845998889-kms/algoseek-marketdata/us-equity-taq-faang/2021/20210104/F/FB.csv.gz
upload: algoseek-marketdata/us-equity-taq-faang/2021/20210104/G/GOOG.csv.gz to s3://kdb-demo-829845998889-kms/algoseek-marketdata/us-equity-taq-faang/2021/20210104/G/GOOG.csv.gz
upload: algoseek-marketdata/us-equity-taq-faang/2021/20210104/N/NFLX.csv.gz to s3://kdb-demo-829845998889-kms/algoseek-marketdata/us-equity-taq-faang/2021/20210104/N/NFLX.csv.gz
upload: algoseek-marketdata/us-equity-taq-faang/2021/20210104/A/AAPL.csv.gz to s3://kdb-demo-829845998889-kms/algoseek-marketdata/us-equity-taq-faang/2021/20210104/A/AAPL.csv.gz
upload: algoseek-marketdata/us-equity-taq-faang/2021/20210105/A/AAPL.csv.gz to s3://kdb-demo-829845998889-kms/algo

0

# Before State of HDB
This is the state/contents of the HDB before we process the CSV files as new date. There will be a table (taq) with no data.

In [11]:
# Query the HDB for before state
hdb = get_pykx_connection(client, 
                          environmentId=ENV_ID, clusterName=HDB_CLUSTER_NAME, 
                          userName=KDB_USERNAME, boto_session=session)

tables = hdb("tables[]").py()

# inventory of tables in the database and rows in each
print(80*'=')
print("Tables and Counts")
display( hdb("tables[]!count each value each tables[]") )

# For each table: schema, and samples and counts
for t in tables:
    print(80*'=')
    print (f'Table: {t}')
    print(80*'-')
    display( hdb(f"meta {t}") )
    display( hdb(f"select from {t} where date = min date, i<3") )
    display( hdb(f"select from {t} where date = max date, i<3") )
    display( hdb(f"select rows:count i by date from {t}") )
    display( hdb(f"select rows:count i by date,Ticker from {t}") )


Tables and Counts


,
,
taq,0


Table: taq
--------------------------------------------------------------------------------


,t,f,a
c,,,
date,"""d""",,
Ticker,"""s""",,p
Timestamp,"""n""",,
EventType,"""s""",,
Price,"""f""",,
Quantity,"""j""",,
Exchange,"""s""",,
Conditions,"""s""",,


,date,Ticker,Timestamp,EventType,Price,Quantity,Exchange,Conditions


,date,Ticker,Timestamp,EventType,Price,Quantity,Exchange,Conditions


,rows
date,


,,rows
date,Ticker,


# Process from Cluster
Connect to the GP cluster and use it to process the list of external S3 files (csv.gz format). The list of S3 files to process will be given to the cluster and processed with the process_s3_csvgz function in q provided by this notebook.

## parse_csvgz
This function will process a a local file (csv.gz) into an in-memory table taking a local file and schema as arguments.

## process_s3_csvgz
This function will process an S3 based file (csv.gz) into an in-memory table, levering the parse_csvgz function. The function will copy the S3 file to a working directory (arguments of function), process it with parse_csvgz and clean up after itself by deleting the copied file.

In [12]:
gp = get_pykx_connection(client, 
                        environmentId=ENV_ID, clusterName=CLUSTER_NAME, 
                        userName=KDB_USERNAME, boto_session=session)

In [13]:
# Define functions on cluster
gp('''
diR:{$[11h=type d:key x;raze x,.z.s each` sv/:x,/:d;d]};
nuke:hdel each desc diR@;

parse_csvgz:{[schema;file] (schema;enlist csv) 0: .Q.gz "c"$read1 hsym `$ string file};

process_s3_csvgz:{[schema;work_dir;s3_object]
    r:.aws.s3.get_object[s3_object;work_dir];
    raze {t:parse_csvgz[x;`$y]; hdel hsym`$y; t}[schema] each r`containerFileDestinationPath
};
''')

pykx.Identity(pykx.q('::'))

In [18]:
# search for S3 objects
s3_search=f'{S3_DATA_HOME}/{DATE.year}/{DATE.strftime("%Y%m%d")}/*/*.csv.gz'

# Get list of S3 objects that will be processed into the table
slist = wr.s3.list_objects(s3_search)

# Tables in memory
print( f"Server Tables: {gp('tables[]').py()}" )

# Number of available slave threads (used by peach)
s=gp('\s').py()
print( f"Slave Threads: {s}" )

# Number of files to be processed
print(f"S3 Objects: {len(slist)}")
display(slist[:3])
# STEP if list is empty
if (len(slist) == 0):
    raise SystemExit(f"Stop no files: {s3_search}")

Server Tables: ['taq']
Slave Threads: 4
S3 Objects: 5


['s3://kdb-demo-829845998889-kms/algoseek-marketdata/us-equity-taq-faang/2021/20210105/A/AAPL.csv.gz',
 's3://kdb-demo-829845998889-kms/algoseek-marketdata/us-equity-taq-faang/2021/20210105/A/AMZN.csv.gz',
 's3://kdb-demo-829845998889-kms/algoseek-marketdata/us-equity-taq-faang/2021/20210105/F/FB.csv.gz']

In [15]:
# send slist to cluster
gp['slist'] = slist

# send working directory to cluster, this is where each S3 object will be copied to before processing into a table
gp['wd'] = WORKING_DIR

# Process the list of S3 files into one table (df) and Delete the Date column (this will be the partition as date in schema)
gp('''
taq:raze{.Q.gc[]; process_s3_csvgz [x; y; z]}["DNSSFJSS";wd] peach slist;

delete Date from `taq;
''')


pykx.Identity(pykx.q('::'))

In [16]:
# Show what is in the table, sample head and tail 5 rows...
display( gp('select [3] from taq').pd() )
display( gp('select [-3] from taq').pd() )
print()

# Total number of rows
print( f"Rows: {gp('count taq').py():,}" )

,Timestamp,EventType,Ticker,Price,Quantity,Exchange,Conditions
0,0 days 04:00:00.074828586,TRADE,AAPL,130.07,639,ARCA,20000401
1,0 days 04:00:00.077890184,QUOTE BID,AAPL,120.00,100,ARCA,00000001
2,0 days 04:00:00.077890184,QUOTE ASK,AAPL,0.00,0,ARCA,00000001


,Timestamp,EventType,Ticker,Price,Quantity,Exchange,Conditions
0,0 days 19:59:29.696901119,TRADE,NFLX,521.00,1,ARCA,80000401
1,0 days 19:59:44.493541009,QUOTE BID NB,NFLX,520.00,300,ARCA,00000001
2,0 days 19:59:44.493541009,QUOTE ASK NB,NFLX,521.66,200,ARCA,00000001



Rows: 27,360,751


# In Memory Contents
We now have two in memory tables (df1 and df2). One (df1) was sourced from a local csv file, and the other (df2) was sourced from a file on S3 (which was put in S3 from this location).

In [19]:
# tables we have now
print("Tables and Counts")
display( gp("tables[]!count each value each tables[]") )

Tables and Counts


,
,
taq,27360751


# Add to Database: Create the Changeset
Will create a changeset for the database that includes the two tables (df1 and df2) and will create a changeset as a new date partition of the database with the in-memory tables splayed to disk for today's date.

## q Code
Code run on the cluster to save in-memory tables to disk and then add those files to the maanged database. This adds the two in memory tables (df1 and df2) to the Managed Database under a new date partition.

```
diR:{$[11h=type d:key x;raze x,.z.s each` sv/:x,/:d;d]};
nuke:hdel each desc diR@;

saveTables:{[db;path;d]
    .aws.get_latest_sym_file[db;path];
    t:tables`.;
    t@:where `g=attr each t@\:`sym;
    {.Q.dpft[hsym`$x;y;`sym;z]}[path;d] each tables`.;

    dt:string d;
    dict:flip`input_path`database_path`change_type!(
        (`$path,dt;`$path,"sym");
        (`$"/",dt,"/";`$"/");`PUT`PUT);
    cid:.aws.create_changeset[db;dict];
    
    nuke hsym`$path,string[d];
    hdel hsym`$path,"sym";

    @[;`sym;`g#] each t;
    .Q.gc[];

    cid
};

```

In [20]:
# define some functions

# diR: gets listing of files in directory
# nuke: deletes directory and its contents
# saveTables: Saves all in-memory tables that have the group attribute on a sym column, updating the sym file as well.

gp("""
diR:{$[11h=type d:key x;raze x,.z.s each` sv/:x,/:d;d]};
nuke:hdel each desc diR@;

saveTables:{[db;path;d]
    .aws.get_latest_sym_file[db;path];
    t:tables`.;
    t@:where `g=attr each t@\:`Ticker;
    {.Q.dpft[hsym`$x;y;`Ticker;z]}[path;d] each tables`.;

    dt:string d;
    dict:flip`input_path`database_path`change_type!(
        (`$path,dt;`$path,"sym");
        (`$"/",dt,"/";`$"/");`PUT`PUT);
    cid:.aws.create_changeset[db;dict];
    
    nuke hsym`$path,string[d];
    hdel hsym`$path,"sym";

    @[;`Ticker;`g#] each t;
    .Q.gc[];

    cid
};
""")

pykx.Identity(pykx.q('::'))

In [21]:
# tables will be added as a new date partition
gp["dt"] = DATE

# save tables and collect the changeset ID
cmd = f'cid:saveTables["{DB_NAME}";"{WORKING_DIR}/";dt]'
gp(cmd)

# Newly created changset ID
changeset_id = str(gp("cid`id"))
display( changeset_id )

'YMesyucSrX60IrXKdemSzA'

In [22]:
# Wait for the changeset to ingest
wait_for_changeset_status(client, environmentId=ENV_ID, databaseName=DB_NAME, changesetId=changeset_id, show_wait=True)
print("**Done**")

Status is IN_PROGRESS, total wait 0:00:00, waiting 10 sec ...
Status is IN_PROGRESS, total wait 0:00:10, waiting 10 sec ...
Status is IN_PROGRESS, total wait 0:00:20, waiting 10 sec ...
Status is IN_PROGRESS, total wait 0:00:30, waiting 10 sec ...
**Done**


# Update the HDB
Now that the database has been populated with new data, update the HDB's view to reflect the latest changeset_id and query its contents to confirm the data from the CSVs are now in the tables of the database HDB is serving up.

In [23]:
# get the list of changesets in the database
c_set_list = list_kx_changesets(client, environmentId=ENV_ID, databaseName=DB_NAME)

if len(c_set_list) != 0:
    # sort by create time
    c_set_list = sorted(c_set_list, key=lambda d: d['createdTimestamp']) 
    latest_changeset = c_set_list[-1]['changesetId']

    # Check if dataview already exists and is set to the requested changeset_id
    resp = get_kx_dataview(client=client, environmentId=ENV_ID, databaseName=DB_NAME, dataviewName=DBVIEW_NAME)

    if resp is None:
        resp = client.create_kx_dataview(
            environmentId = ENV_ID, 
            databaseName=DB_NAME, 
            dataviewName=DBVIEW_NAME,
            azMode='SINGLE',
            availabilityZoneId=AZ_ID,
            changesetId=latest_changeset, # latest changeset_id
            segmentConfigurations=[
                { 
                    'volumeName': VOLUME_NAME,
                    'dbPaths': ['/*'],  # cache all of database
    #                "onDemand": True,   # cache data onDemand (on read) else will ensure all is cached
                }
            ],
    #        readWrite=True,
            autoUpdate=False,
            description = f'Dataview of database'
        )
    elif resp['changesetId'] != latest_changeset:
        print(f"Dataview {DBVIEW_NAME} exists but needs updating, updating...")
        resp = client.update_kx_dataview(environmentId=ENV_ID, 
            databaseName=DB_NAME, 
            dataviewName=DBVIEW_NAME, 
            changesetId=latest_changeset, 
            segmentConfigurations=[
                {'dbPaths': ['/*'], 'volumeName': VOLUME_NAME}
            ]
        )
    else:
        print(f"Dataview {DBVIEW_NAME} exists with current changeset: {latest_changeset}")
    
else:
    # no changesets, do NOT create view
    print(f"No changeset in database: {DB_NAME}, Dataview {DBVIEW_NAME} not created")        


Dataview DEMO_DB_VIEW exists but needs updating, updating...


In [24]:
# wait for view to be ready
wait_for_dataview_status(client=client, environmentId=ENV_ID, databaseName=DB_NAME, dataviewName=DBVIEW_NAME, show_wait=True)

Dataview: DEMO_DB_VIEW status is UPDATING, total wait 0:00:00, waiting 30 sec ...
Dataview: DEMO_DB_VIEW status is UPDATING, total wait 0:00:30, waiting 30 sec ...
Dataview: DEMO_DB_VIEW status is UPDATING, total wait 0:01:00, waiting 30 sec ...
Dataview: DEMO_DB_VIEW status is UPDATING, total wait 0:01:30, waiting 30 sec ...
Dataview: DEMO_DB_VIEW status is UPDATING, total wait 0:02:00, waiting 30 sec ...
Dataview: DEMO_DB_VIEW status is UPDATING, total wait 0:02:30, waiting 30 sec ...
Dataview: DEMO_DB_VIEW status is UPDATING, total wait 0:03:00, waiting 30 sec ...
Dataview: DEMO_DB_VIEW status is UPDATING, total wait 0:03:30, waiting 30 sec ...
Dataview: DEMO_DB_VIEW status is UPDATING, total wait 0:04:00, waiting 30 sec ...
Dataview: DEMO_DB_VIEW status is UPDATING, total wait 0:04:30, waiting 30 sec ...
Dataview: DEMO_DB_VIEW status is UPDATING, total wait 0:05:00, waiting 30 sec ...
Dataview: DEMO_DB_VIEW status is UPDATING, total wait 0:05:30, waiting 30 sec ...
Dataview: DEMO_D

{'databaseName': 'DEMO_DB',
 'dataviewName': 'DEMO_DB_VIEW',
 'azMode': 'SINGLE',
 'availabilityZoneId': 'use1-az6',
 'changesetId': 'YMesyucSrX60IrXKdemSzA',
 'segmentConfigurations': [{'dbPaths': ['/*'],
   'volumeName': 'DEMO_SHARED_VOLUME',
   'onDemand': False}],
 'activeVersions': [{'changesetId': 'YMesyucSrX60IrXKdemSzA',
   'segmentConfigurations': [{'dbPaths': ['/*'],
     'volumeName': 'DEMO_SHARED_VOLUME',
     'onDemand': False}],
   'attachedClusters': [],
   'createdTimestamp': datetime.datetime(2024, 5, 8, 19, 3, 5, 463000, tzinfo=tzlocal()),
   'versionId': 'gMesyz37zA6bcFdXH7P71w'},
  {'changesetId': 'EMesq6rn0bqzz6vvXlf5Kw',
   'segmentConfigurations': [{'dbPaths': ['/*'],
     'volumeName': 'DEMO_SHARED_VOLUME',
     'onDemand': False}],
   'attachedClusters': ['demo_csv_cluster', 'demo_hdb_cluster'],
   'createdTimestamp': datetime.datetime(2024, 5, 8, 17, 55, 2, 963000, tzinfo=tzlocal()),
   'versionId': 'FMesrBhZzqITXJkjmRJR7w'}],
 'description': 'Dataview of data

In [25]:
# Update the HDB Cluster to use updated view of database
resp=client.update_kx_cluster_databases(environmentId=ENV_ID, 
    clusterName=HDB_CLUSTER_NAME, 
    databases=[
        {'databaseName': DB_NAME, 'dataviewName': DBVIEW_NAME}
    ],
    deploymentConfiguration={
        'deploymentStrategy': 'ROLLING'
    }
)

In [26]:
# Wait for the HDB to update
wait_for_cluster_status(client, environmentId=ENV_ID, clusterName=HDB_CLUSTER_NAME, show_wait=True)

Cluster: demo_hdb_cluster status is UPDATING, total wait 0:00:00, waiting 30 sec ...
Cluster: demo_hdb_cluster status is UPDATING, total wait 0:00:30, waiting 30 sec ...
Cluster: demo_hdb_cluster status is UPDATING, total wait 0:01:00, waiting 30 sec ...
Cluster: demo_hdb_cluster status is UPDATING, total wait 0:01:30, waiting 30 sec ...
Cluster: demo_hdb_cluster status is UPDATING, total wait 0:02:00, waiting 30 sec ...
Cluster: demo_hdb_cluster status is UPDATING, total wait 0:02:30, waiting 30 sec ...
Cluster: demo_hdb_cluster status is UPDATING, total wait 0:03:00, waiting 30 sec ...
Cluster: demo_hdb_cluster status is UPDATING, total wait 0:03:30, waiting 30 sec ...
Cluster: demo_hdb_cluster status is UPDATING, total wait 0:04:00, waiting 30 sec ...
Cluster: demo_hdb_cluster status is UPDATING, total wait 0:04:30, waiting 30 sec ...
Cluster: demo_hdb_cluster status is UPDATING, total wait 0:05:00, waiting 30 sec ...
Cluster: demo_hdb_cluster status is UPDATING, total wait 0:05:30,

{'status': 'RUNNING',
 'clusterName': 'demo_hdb_cluster',
 'clusterType': 'HDB',
 'volumes': [{'volumeName': 'DEMO_SHARED_VOLUME', 'volumeType': 'NAS_1'}],
 'databases': [{'databaseName': 'DEMO_DB',
   'dataviewConfiguration': {'dataviewName': 'DEMO_DB_VIEW',
    'dataviewVersionId': 'gMesyz37zA6bcFdXH7P71w',
    'changesetId': 'YMesyucSrX60IrXKdemSzA',
    'segmentConfigurations': [{'dbPaths': ['/*'],
      'volumeName': 'DEMO_SHARED_VOLUME',
      'onDemand': False}]}}],
 'clusterDescription': 'Created with create_all notebook',
 'releaseLabel': '1.0',
 'vpcConfiguration': {'vpcId': 'vpc-0fe2b9c50f3ad382f',
  'securityGroupIds': ['sg-0c99f1cfb9c3c7fd9'],
  'subnetIds': ['subnet-04052219ec25b062b'],
  'ipAddressType': 'IP_V4'},
 'commandLineArguments': [{'key': 's', 'value': '4'}],
 'executionRole': 'arn:aws:iam::829845998889:role/kdb-all-user',
 'lastModifiedTimestamp': datetime.datetime(2024, 5, 8, 19, 15, 46, 726000, tzinfo=tzlocal()),
 'azMode': 'SINGLE',
 'availabilityZoneId': 'u

# Query the HDB 
Show the new data by querying the tables in the HDB. The data has changed from the data that was just added to the database.

In [27]:
# Query the HDB for before state
hdb = get_pykx_connection(client, 
                          environmentId=ENV_ID, clusterName=HDB_CLUSTER_NAME, 
                          userName=KDB_USERNAME, boto_session=session)

tables = hdb("tables[]").py()

# inventory of tables in the database and rows in each
print(80*'=')
print("Tables and Counts")
display( hdb("tables[]!count each value each tables[]") )

# For each table: schema, and samples and counts
for t in tables:
    print(80*'=')
    print (f'Table: {t}')
    print(80*'-')
    display( hdb(f"meta {t}") )
    display( hdb(f"select from {t} where date = min date, i<5") )
    display( hdb(f"select from {t} where date = max date, i<5") )
    display( hdb(f"select rows:count i by date from {t}") )
    display( hdb(f"select rows:count i by date,Ticker from {t}") )


Tables and Counts


,
,
taq,27360751


Table: taq
--------------------------------------------------------------------------------


,t,f,a
c,,,
date,"""d""",,
Ticker,"""s""",,p
Timestamp,"""n""",,
EventType,"""s""",,
Price,"""f""",,
Quantity,"""j""",,
Exchange,"""s""",,
Conditions,"""s""",,


,date,Ticker,Timestamp,EventType,Price,Quantity,Exchange,Conditions
,,,,,,,,
0,2021.01.05,AAPL,0D04:00:00.074828586,TRADE,130.07,639,ARCA,20000401
1,2021.01.05,AAPL,0D04:00:00.077890184,QUOTE BID,120f,100,ARCA,00000001
2,2021.01.05,AAPL,0D04:00:00.077890184,QUOTE ASK,0f,0,ARCA,00000001
3,2021.01.05,AAPL,0D04:00:00.077890184,QUOTE BID NB,120f,100,ARCA,00000001
4,2021.01.05,AAPL,0D04:00:00.077938515,QUOTE BID,123f,100,ARCA,00000001


,date,Ticker,Timestamp,EventType,Price,Quantity,Exchange,Conditions


,rows
date,
2021.01.05,27360751


pykx.KeyedTable(pykx.q('
date       Ticker| rows    
-----------------| --------
2021.01.05 AAPL  | 21186507
2021.01.05 AMZN  | 1685421 
2021.01.05 FB    | 1720887 
2021.01.05 GOOG  | 1550965 
2021.01.05 NFLX  | 1216971 
'))

In [28]:
print( f"Last Run: {datetime.datetime.now()}" )

Last Run: 2024-05-08 19:16:17.983785
